![](https://drive.google.com/uc?export=view&id=1z-VzwrzVfh8ZFG6p3pXFJHxXF_cOtMC6)
# **Sequence Models Workshop for OxML 2020** 

---

### **Prepared by *Asmita Poddar* & *Piotr Kozakowski***


In [ ]:
#!pip install torch torchtext
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Language Modeling**

Language Modelling is the core problem for a number of of Natural Language Processing (NLP) tasks such as speech to text, conversational system, and text summarization.

In this tutorial, you will learn how to create a language model for ***natural language text generation*** by implementing and training **Long Short Term Memory (LSTM)** networks and **Transformers**. 

A trained language model learns the likelihood of occurrence of a word based on the previous sequence of words used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level.

After completing this tutorial, you will know:


*   The challenge of developing a good framing of a word-based language model for a corpus of text using *PyTorch*
*   How to generate sequences using a fit language model.





In [ ]:
import math
import pandas as pd
import re
import numpy as np
import random
import time
import tqdm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets

## Data

The [WikiText language modeling dataset](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) is a collection of over 100 million tokens extracted from the set of verified `Good` and `Featured` articles on Wikipedia.

PyTorch provides many inbuilt datasets for many applications as well as tools for data processing. 

In this tutorial, we will use the Pytorch Dataset - [wikitext2](https://torchtext.readthedocs.io/en/latest/datasets.html#wikitext-2). 

The texts in this dataset have already been tokenized to word level, and split into train, validation, test sets. We further split the data into batches for model training.

In [ ]:
field = data.Field()
(train, val, test) = datasets.language_modeling.WikiText2.splits(text_field=field)

train_data = next(iter(train))
val_data = next(iter(val))
test_data = next(iter(test))

print('# Train tokens: ', len(train_data.text))  
print('# Val tokens: ', len(val_data.text))      
print('# Test tokens: ', len(test_data.text))    

# Train tokens:  2088628
# Val tokens:  217646
# Test tokens:  245569


In [ ]:
# We will need to know the size of the vocabulary later for
#  both defining the word embedding layer in the model,
# and for encoding output words.
field.build_vocab(train, vectors=None)
print('Vocabulary size:', len(field.vocab))

Vocabulary size: 33279


In [ ]:
BATCH_SIZE = 128
SEQ_LEN = 40
# Split the data into batches of size BATCH_SIZE 
(train, val, test) = data.BPTTIterator.splits((train, val, test), batch_size=BATCH_SIZE, bptt_len=SEQ_LEN)

In [ ]:
batch = next(iter(train))
print('Batch text shape: ', batch.text.shape)     # [seq_len x batch_size]
print('Batch text: \n', batch.text)               #  consecutive words are in rows of the tensor
print('Batch target: \n', batch.target)           # target is the same as text, but shifted one position up

Batch text shape:  torch.Size([40, 128])
Batch text: 
 tensor([[    9,  1080,    38,  ..., 20496,     5,    28],
        [   11,   343, 24830,  ...,    21,     2,     2],
        [ 3932,     5,     3,  ...,  7154,  4917,   508],
        ...,
        [  955,  2531,    22,  ...,     6,    42,    27],
        [    3,  1500,    27,  ...,  2661,   247,  1530],
        [   25,   637, 22950,  ...,  1689,  1509,     0]])
Batch target: 
 tensor([[   11,   343, 24830,  ...,    21,     2,     2],
        [ 3932,     5,     3,  ...,  7154,  4917,   508],
        [ 4429,     2,    56,  ...,     0,  7129,     5],
        ...,
        [    3,  1500,    27,  ...,  2661,   247,  1530],
        [   25,   637, 22950,  ...,  1689,  1509,     0],
        [   10,   586,     3,  ...,    45,    11,    13]])


# Long Short Term Memory (LSTM) Networks
Recurrent Neural Networks suffer from two problems: 
* vanishing gradient and exploding gradient
* inability to capture long-term dependencies

LSTM solve this issue by explicitly introducing a memory unit, called the cell into the network. Read more at: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Therefore, this single unit makes decision by considering the current input, previous output and previous memory. And it generates a new output and alters its memory.

Layers in the LSTM model:
1. **Input Layer**: Takes the sequence of words as input
2. **Embedding Layer**: Creates embeddings from the input sequence using a simple lookup table that stores embeddings of a fixed dictionary and size
3. **LSTM Layer**: Computes the output using LSTM units. I have added 128 units in the layer, but this number can be fine tuned later.
4. **Dropout Layer**: A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting.
5. **Output Layer** : Computes the probability of the best possible next word as output


### Model


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = len(field.vocab)

## TODO: Experiment with for fine-tuning the model
EMB_SIZE = 256
NHIDDEN = 128
NLAYERS = 3
DROPOUT = 0.5
EPOCHS = 3
LR = 0.001

![](https://drive.google.com/uc?export=view&id=1JMR622gBqx5U5jZPQLxM1FxAKmscKeM-)



In [ ]:
class LSTMModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, vocab_size, emb_size, nhidden, nlayers, dropout=0.5):
        super(LSTMModel, self).__init__()
        
        self.vocab_size = vocab_size
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(emb_size, nhidden, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhidden, vocab_size)

        self.init_weights()
        self.nhidden = nhidden
        self.nlayers = nlayers

    # Initialise weights for encoder and decoder (uniform initialisation)
    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

     # Initialise weights for hidden $ cell units with zeros
    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhidden),
                    weight.new_zeros(self.nlayers, bsz, self.nhidden))

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.lstm(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.vocab_size)
        return F.log_softmax(decoded, dim=1), hidden

   

### Model Training

##### **Model:** LSTM

##### **Loss Function:** Negative Log-Likelihood (NLL):  $L(y) = −log(y)$

[](https://ljvmiranda921.github.io/assets/png/cs231n-ann/neg_log.png)

##### **Optimizer:** Adam Optimizer

In [ ]:
model = LSTMModel(VOCAB_SIZE, EMB_SIZE, NHIDDEN, NLAYERS, DROPOUT).to(device)
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

print(model)

LSTMModel(
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(33279, 256)
  (lstm): LSTM(256, 128, num_layers=3, dropout=0.5)
  (decoder): Linear(in_features=128, out_features=33279, bias=True)
)


In [ ]:
eval_it = iter(val)


# Run training on one batch
def run_on_batch(batch):
  text = batch.text.cuda()
  target = batch.target.cuda()
  target = target.view(-1)
  hidden = model.init_hidden(BATCH_SIZE)

  # Starting each batch, we detach the hidden state from how it was previously produced.
  # If we didn't, the model would try backpropagating all the way to start of the dataset.
  hidden = (hidden[0].detach(), hidden[1].detach())
  pred, hidden = model(text, hidden)
  return loss_fn(pred, target)

for epoch in range(EPOCHS):
  print('**** Running Epoch {} ****'.format(epoch))

  for (i, batch) in enumerate(train):
    
    if batch.text.shape[1] != BATCH_SIZE:
      #Skip an incomplete batch.
      print('Skipping incomplete batch {} ...'.format(i))
      continue

    # Turn on training mode which enables dropout.
    model.train()
    model.zero_grad()

    train_loss = run_on_batch(batch)
    train_loss.backward()

    # `clip_grad_norm` helps prevent the exploding gradient problem in LSTMs.
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # max norm of the gradients = 1.0
    optimizer.step()
    
    if i % 400 == 0:
      print('Batch {} ----'.format(i) )
      print('Train loss: ', float(train_loss) )

      # Turn on evaluation mode which disables dropout.
      model.eval()
      eval_loss = float(run_on_batch(next(eval_it)))
      print('Eval loss: ',eval_loss )

torch.save(model, 'model.pt')

# Download model from Colab
from google.colab import files
files.download("model.pt")

**** Running Epoch 0 ****
Batch 0 ----
Train loss:  6.0173773765563965
Eval loss:  5.884695529937744
**** Running Epoch 1 ****
Batch 0 ----
Train loss:  5.95809268951416
Eval loss:  5.77343225479126
**** Running Epoch 2 ****
Batch 0 ----
Train loss:  5.909502983093262
Eval loss:  5.8311262130737305


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Text Generation 
 

Now, we are ready to generate text. 

We begin with some input (seed) word and use the trained model to predict the next word based on the input words. Then, we append the predicted word into the input, and have the model predict the next word and so on. We continue the process until we obtain a sequence with the length we want (here we generate 20 words). 

The multiple generated words can be appended together to get the generated sequence!

 

In [ ]:
example = next(eval_it).text.cuda()
buffer = example.clone()
input = torch.tensor([[example[0][0]]]).to(device)

### TODO: Your own input
#input = torch.randint(VOCAB_SIZE, (1, 1), dtype=torch.long).to(device). #generating a random starting word
### TODO: Specify no. of words to be generated 
NO_WORDS = 20

In [ ]:
# Load saved trained model
with open('model.pt', 'rb') as f:
    model = torch.load(f).to(device)
    
generated_text = ''

# Turn on evaluation mode which disables dropout.
model.eval()
hidden = model.init_hidden(1)

for i in range(NO_WORDS):
  output, hidden = model(input, hidden)
  word_weights = output.squeeze().exp().cpu()
  word_idx = torch.multinomial(word_weights, 1)[0]
  input.fill_(word_idx)   # current word is input to next hidden unit
  generated_text = generated_text + field.vocab.itos[word_idx]+ ' '

dataset_text = [field.vocab.itos[idx] for idx in buffer[:, 0]]
print('INPUT WORD: ', field.vocab.itos[buffer[0][0]])
print('DATASET PORTION:', dataset_text[: NO_WORDS])
print('GENERATED TEXT: ', generated_text)

INPUT WORD:  before
DATASET PORTION: ['before', 'hatching', 'into', '<unk>', 'larvae', '.', 'Homarus', 'gammarus', 'is', 'a', 'highly', 'esteemed', 'food', ',', 'and', 'is', 'widely', 'caught', 'using', 'lobster']
GENERATED TEXT:  that meanwhile and earn FOCA after d , the line started that the tenure , <unk> of overseas and several 


# Transformers

Define the model:

In [ ]:
# from https://github.com/pytorch/examples/blob/master/word_language_model/model.py

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class TransformerLM(nn.Module):

  def __init__(
      self,
      vocab_size,
      n_layers=3,
      n_heads=4,
      d_model=256,
      d_ff=512,
      dropout=0.5,
      activation='relu',
  ):
    super().__init__()
    
    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoder = PositionalEncoding(d_model, dropout)
    encoder_layer = nn.TransformerEncoderLayer(d_model, n_heads, d_ff, dropout, activation)
    encoder_norm = nn.LayerNorm(d_model)
    self.encoder = nn.TransformerEncoder(encoder_layer, n_layers, encoder_norm)
    self.output = nn.Linear(d_model, vocab_size)
    self.output.weight = self.embedding.weight
    self.log_softmax = nn.LogSoftmax(dim=-1)

    self.init_weights()

    self._d_model = d_model
    self._mask = None
    self._memory = None

  def init_weights(self):
    initrange = 0.1
    nn.init.uniform_(self.embedding.weight, -initrange, initrange)
    nn.init.zeros_(self.output.weight)
    nn.init.uniform_(self.output.weight, -initrange, initrange)

  def _generate_square_subsequent_mask(self, sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

  def forward(self, x):
    if self._mask is None:
      self._mask = self._generate_square_subsequent_mask(x.size(0)).to(x.device)

    x = self.embedding(x) * math.sqrt(self._d_model)
    x = self.pos_encoder(x)
    x = self.encoder(x, self._mask)
    x = self.output(x)
    return self.log_softmax(x)

Prepare the data:

In [ ]:
batch_size = 128
seq_len = 32

field = data.Field()
(train, val, test) = datasets.language_modeling.WikiText2.splits(text_field=field)
field.build_vocab(train, max_size=None)
(train, val, test) = data.BPTTIterator.splits(
    (train, val, test), batch_size=batch_size, bptt_len=seq_len, repeat=True, shuffle=True
)

Train the model:

In [ ]:
model = TransformerLM(vocab_size=len(field.vocab)).cuda()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

eval_it = iter(val)

def run_on_batch(batch):
  text = batch.text.cuda()
  target = batch.target.cuda()
  pred = model(text).permute(0, 2, 1)
  return loss_fn(pred, target)

for (i, batch) in tqdm.tqdm(enumerate(train)):
  if batch.text.size() != (seq_len, batch_size):
    # Skip an incomplete batch.
    continue

  optimizer.zero_grad()
  loss = run_on_batch(batch)
  loss.backward()
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  optimizer.step()

  if i % (len(train) // 10) == 0:
    print('train loss:', float(loss))
    model.eval()
    print('eval loss:', float(run_on_batch(next(eval_it))))
    model.train()

  if i == 3000:
    break


0it [00:00, ?it/s]

train loss: 10.744112014770508



1it [00:01,  1.19s/it]

eval loss: 9.901606559753418



2it [00:01,  1.05it/s]
3it [00:01,  1.27it/s]
4it [00:02,  1.49it/s]
5it [00:02,  1.70it/s]
6it [00:03,  1.88it/s]
7it [00:03,  2.03it/s]
8it [00:03,  2.15it/s]
9it [00:04,  2.25it/s]
10it [00:04,  2.32it/s]
11it [00:05,  2.38it/s]
12it [00:05,  2.42it/s]
13it [00:05,  2.46it/s]
14it [00:06,  2.48it/s]
15it [00:06,  2.51it/s]
16it [00:07,  2.53it/s]
17it [00:07,  2.54it/s]
18it [00:07,  2.54it/s]
19it [00:08,  2.54it/s]
20it [00:08,  2.54it/s]
21it [00:09,  2.54it/s]
22it [00:09,  2.54it/s]
23it [00:09,  2.53it/s]
24it [00:10,  2.52it/s]
25it [00:10,  2.52it/s]
26it [00:11,  2.52it/s]
27it [00:11,  2.54it/s]
28it [00:11,  2.55it/s]
29it [00:12,  2.55it/s]
30it [00:12,  2.55it/s]
31it [00:13,  2.55it/s]
32it [00:13,  2.53it/s]
33it [00:13,  2.53it/s]
34it [00:14,  2.53it/s]
35it [00:14,  2.52it/s]
36it [00:15,  2.53it/s]
37it [00:15,  2.53it/s]
38it [00:15,  2.53it/s]
39it [00:16,  2.54it/s]
40it [00:16,  2.55it/s]
41it [00:16,  2.54it/s]
42it [00:17,  2.55it/s]
43it [00:17,  2.55it/s]

...


1838it [12:12,  2.34it/s]
1839it [12:12,  2.39it/s]
1840it [12:13,  2.43it/s]
1841it [12:13,  2.46it/s]
1842it [12:14,  2.49it/s]
1843it [12:14,  2.51it/s]
1844it [12:14,  2.52it/s]
1845it [12:15,  2.53it/s]
1846it [12:15,  2.53it/s]
1847it [12:15,  2.52it/s]
1848it [12:16,  2.51it/s]
1849it [12:16,  2.52it/s]
1850it [12:17,  2.51it/s]
1851it [12:17,  2.50it/s]
1852it [12:17,  2.50it/s]
1853it [12:18,  2.51it/s]
1854it [12:18,  2.51it/s]
1855it [12:19,  2.52it/s]
1856it [12:19,  2.52it/s]
1857it [12:19,  2.52it/s]
1858it [12:20,  2.51it/s]
1859it [12:20,  2.51it/s]
1860it [12:21,  2.52it/s]
1861it [12:21,  2.53it/s]
1862it [12:21,  2.54it/s]
1863it [12:22,  2.55it/s]
1864it [12:22,  2.55it/s]
1865it [12:23,  2.54it/s]
1866it [12:23,  2.54it/s]
1867it [12:23,  2.54it/s]
1868it [12:24,  2.53it/s]
1869it [12:24,  2.52it/s]
1870it [12:25,  2.51it/s]
1871it [12:25,  2.52it/s]
1872it [12:25,  2.52it/s]
1873it [12:26,  2.52it/s]
1874it [12:26,  2.52it/s]
1875it [12:27,  2.53it/s]
1876it [12:

train loss: 5.465083599090576
eval loss: 5.492844581604004



1889it [12:32,  2.31it/s]
1890it [12:33,  2.38it/s]
1891it [12:33,  2.43it/s]
1892it [12:33,  2.45it/s]
1893it [12:34,  2.47it/s]
1894it [12:34,  2.49it/s]
1895it [12:35,  2.49it/s]
1896it [12:35,  2.50it/s]
1897it [12:35,  2.51it/s]
1898it [12:36,  2.52it/s]
1899it [12:36,  2.52it/s]
1900it [12:37,  2.52it/s]
1901it [12:37,  2.53it/s]
1902it [12:37,  2.54it/s]
1903it [12:38,  2.54it/s]
1904it [12:38,  2.55it/s]
1905it [12:39,  2.55it/s]
1906it [12:39,  2.55it/s]
1907it [12:39,  2.54it/s]
1908it [12:40,  2.54it/s]
1909it [12:40,  2.54it/s]
1910it [12:41,  2.54it/s]
1911it [12:41,  2.53it/s]
1912it [12:41,  2.52it/s]
1913it [12:42,  2.52it/s]
1914it [12:42,  2.51it/s]
1915it [12:43,  2.52it/s]
1916it [12:43,  2.53it/s]
1917it [12:43,  2.54it/s]
1918it [12:44,  2.55it/s]
1919it [12:44,  2.55it/s]
1920it [12:45,  2.54it/s]
1921it [12:45,  2.54it/s]
1922it [12:45,  2.54it/s]
1923it [12:46,  2.53it/s]
1924it [12:46,  2.53it/s]
1925it [12:47,  2.53it/s]
1926it [12:47,  2.52it/s]
1927it [12:

train loss: 5.454954147338867
eval loss: 5.440124034881592



1940it [12:53,  2.31it/s]
1941it [12:53,  2.38it/s]
1942it [12:53,  2.43it/s]
1943it [12:54,  2.46it/s]
1944it [12:54,  2.48it/s]
1945it [12:55,  2.50it/s]
1946it [12:55,  2.50it/s]
1947it [12:55,  2.51it/s]
1948it [12:56,  2.52it/s]
1949it [12:56,  2.52it/s]
1950it [12:57,  2.52it/s]
1951it [12:57,  2.52it/s]
1952it [12:57,  2.53it/s]
1953it [12:58,  2.54it/s]
1954it [12:58,  2.55it/s]
1955it [12:59,  2.55it/s]
1956it [12:59,  2.55it/s]
1957it [12:59,  2.55it/s]
1958it [13:00,  2.54it/s]
1959it [13:00,  2.53it/s]
1960it [13:00,  2.53it/s]
1961it [13:01,  2.53it/s]
1962it [13:01,  2.51it/s]
1963it [13:02,  2.51it/s]
1964it [13:02,  2.51it/s]
1965it [13:02,  2.51it/s]
1966it [13:03,  2.51it/s]
1967it [13:03,  2.51it/s]
1968it [13:04,  2.52it/s]
1969it [13:04,  2.53it/s]
1970it [13:04,  2.53it/s]
1971it [13:05,  2.54it/s]
1972it [13:05,  2.55it/s]
1973it [13:06,  2.55it/s]
1974it [13:06,  2.54it/s]
1975it [13:06,  2.54it/s]
1976it [13:07,  2.54it/s]
1977it [13:07,  2.52it/s]
1978it [13:

In [ ]:
example = next(eval_it).text.cuda()
buffer = example.clone()
model.eval()
for i in range(example.size(0) // 2, example.size(0)):
  pred = model(buffer)
  buffer[i] = torch.multinomial(torch.exp(pred[i - 1, 0]), 1)

text = [field.vocab.itos[idx] for idx in buffer[:, 0]]
print('dataset portion:', text[: example.size(0) // 2])
print('predicted portion:', text[example.size(0) // 2 :])

dataset portion: ['is', 'a', 'highly', '<unk>', 'food', ',', 'and', 'is', 'widely', 'caught', 'using', 'lobster', '<unk>', ',', 'mostly', 'around']
predicted portion: ['any', '<unk>', 'and', 'other', 'methods', 'of', 'poor', 'and', 'hold', 'that', 'are', 'treated', 'to', 'a', 'person', '.']


In [ ]:
|

['Hutchings',
 '(',
 'Aniston',
 'of',
 'University',
 'with',
 'in',
 'of',
 'urgent',
 'signal',
 '<unk>',
 'seller',
 ',',
 ',',
 'force',
 "'s",
 'she',
 'television',
 '!',
 'could',
 '.',
 ',',
 'package',
 'and',
 'Disciplina',
 ')',
 ',',
 '"',
 'had',
 ',',
 ',',
 '.',
 'founded',
 'in',
 '<unk>']

Adapted from [PyTorch](https://github.com/pytorch/examples/tree/master/word_language_model)